In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("./data/MSCallGraph/MSCallGraph_0.csv")

In [4]:
df.head()

,Unnamed: 0,traceid,timestamp,rpcid,um,rpctype,dm,interface,rt
0,0,0b133c1915919238193454000e5d37,219678,0.1.3.1.1.1.12,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,0
1,1,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.19,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
2,2,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.18,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
3,3,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.4,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
4,4,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.17,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1


In [5]:
traceids = df['traceid'].unique()

In [6]:
def tracegraph(index=0, save=False):
    tdf = df[df['traceid']==traceids[index]] # trace data frame 
    tdf = tdf.sort_values(by="rpcid", key=lambda col: col.str.split(".").str.len()) # to order from surface down
    print(f"Total calls: {len(tdf)}")
    print(f"Total unique DMs: {len(tdf['dm'].unique())}")
    class Node:
        def __init__(self, id):
            self.id = id
            self.calls = []
    nodes = {}
    for i, (idx, row) in enumerate(tdf.iterrows()):
        um = row["um"]
        dm = row["dm"]
        if um == "(?)" or str(um).lower() == "nan":
            nodes[dm] = Node(dm)
        else:
            if um not in nodes:
                nodes[um] = Node(um)
            if dm not in nodes and not (dm == "(?)" or str(dm).lower() == "nan"):
                nodes[dm] = Node(dm)
            if dm in nodes:
                nodes[um].calls.append(nodes[dm])

    print(f"Total Nodes: {len(nodes)}")
    print("Note: expect off by 1 or 2 because of nan and (?)")

    node_id_to_index = {node_id: idx for idx, node_id in enumerate(nodes.keys())}

        
    graph = nx.DiGraph() 

    for node_id, node in nodes.items():
        graph.add_node(node_id_to_index[node_id])
    for node_id, node in nodes.items():
        for call in node.calls:
            graph.add_edge(node_id_to_index[node_id], node_id_to_index[call.id])
    
    plt.figure(figsize=(13, 13))
    pos = nx.spring_layout(graph)  
    nx.draw_networkx_nodes(graph, pos, node_size=600, node_color="lightblue")
    nx.draw_networkx_edges(graph, pos, arrowstyle="->", arrowsize=20)
    nx.draw_networkx_labels(graph, pos, font_size=10, font_color="black")
    
    plt.title("Microservice Call Graph")
    if save:
        plt.savefig(f"tracegraph+{index}.png")
    plt.show()

In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(tracegraph, index=widgets.IntSlider(min=0, max=len(traceids)-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='index', max=130511), Checkbox(value=False, description='…